### Project Description
Estimate the value of houses in Ames, Iowa, using 79 explanatory variables. The dataset is fairly small, with only 1460 training samples. The 79 features in the dataset are a mix of categorical and numerical features, and leave a lot of room for feature engineering.

This notebook merely preprocesses the data.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore

In [ ]:
# set keggle on True if you want data for keggle competition
# else leave on keggle on False for normal data
keggle = True

In [ ]:
# import data
data = pd.read_csv('data/AmesHousing.csv')
test_data = pd.read_csv('data/test.csv')
train_data = pd.read_csv('data/train.csv')

data.head()

### OVERVIEW

There are multiple types of variables. Integers, floats, strings, NaN's. Use df.info() to gain detailed insight.


In [ ]:
# display data information
data.info()

There are 2930 entries, numbered 0 to 2929, with **81 different features**. The **target feature** is 'SalePrice'. 

Out of all the columns there are several that have missing values. For instance, 'Alley', 'Pool QC', 'Fence', and 'Misc Feature'. These need to be fixed. 

### Cleaning the data

First, the ourliers are checked by plotting the general living area with sale price, as is mentioned in the documentation of the dataset.

In [ ]:
sns.scatterplot(data['Gr Liv Area'], data['SalePrice'])
plt.show()

Garage year built is also plotted, since this shows also an outlier that is not possible

In [ ]:
# plotting the garage year built to show outliers
sns.scatterplot(data['Garage Yr Blt'], data['SalePrice'])
plt.show()

There are 5 ourliers visible that can be removed by removing datasamples with a general living area greater than 4000.

In [ ]:
mask = data['Gr Liv Area'] > 4000
drop_sample = data[mask]
data = data.drop(drop_sample.index, axis = 0)


Additionally, the value for the garage that is build after 2200 is replaces by NaN. 

In [ ]:
mask = data['Garage Yr Blt'] > 2010
outlier_garage = data['Garage Yr Blt'][mask]
data['Garage Yr Blt'][outlier_garage.index] = np.nan



In [ ]:
# drop column 'Order' since it is unique for each house
clean_data = data.drop('Order', axis=1)

### Dealing with categorical features who are classified as numeric features
Some features in this dataset are stored as numeric data, but are actually categorical features. Therefore, the numeric data is converted into strings so they can be considered as categorical features.

The only feature that is wrongly classified is: MS SubClass (the building class)

In [ ]:
clean_data["MS SubClass"] = clean_data["MS SubClass"].apply(str)


### Changing ordinal categorical features to numerical features

1. A function is made that can change ordinal categorical data to numerical features
1. Dictonaries are created with the categories within features as keys and the corresponding numerical value as dictonary value
3. Lists are created with the features that need to be replaced 


In [ ]:
def ordinal_to_num(lists, dicts):
    
    """This function takes a list and a dictonary. 
    It replaces the categories in a feature to a nummerical """
    
    # loop over the features in the list
    for feature in lists:
        
        # replace the categories for the feature with the value of the dictonary
        if feature in clean_data:
            clean_data[feature] = clean_data[feature].map(dicts)


In [ ]:
# create the list and dictonary for the most common categories
common_ordinal_dict = {'Ex':5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
common_list_num_cats = ['Exter Qual', 'Exter Cond', 'Bsmt Cond', 'Heating QC', 'Kitchen Qual', 'Fireplace Qu', 'Garage Qual', 'Garage Cond', 'Bsmt Qual', 'Bsmt Qual']

# change these to numerical features
ordinal_to_num(common_list_num_cats, common_ordinal_dict)

# create list and dict for Pool QC 
PoolQC_list = ['Pool QC']
PoolQC_dict = {'Ex':4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'NA': 0}

# execute the change for Pool QC
ordinal_to_num(PoolQC_list, PoolQC_dict)

# create list and dict for Land Slope
slope_list = ['Land Slope']
slope_dict = {'Gtl': 1, 'Mod': 2, 'Sev': 3}

# execute the change for Land slope
ordinal_to_num(slope_list, slope_dict)

# create list and dict for Bsmt Exposure
Bsmt_exposure_list = ['Bsmt Exposure']
Bsmt_exposure_dict = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0}

# execute the change for Bsmt Exposure
ordinal_to_num(Bsmt_exposure_list, Bsmt_exposure_dict)

# create list and dict of the types of finished basement
bsmtFin_list = ['BsmtFin Type 1', 'BsmtFin Type 2']
bsmtFin_dict = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0}

# create list and dict of the types of unfinished basement
ordinal_to_num(bsmtFin_list, bsmtFin_dict)

display(clean_data)

In [ ]:
# get overview of missing values
clean_data.isnull()

In [ ]:
def missing_data(df):
    ''' Calculates and returns the percentage of missing data per feature. '''
    percentage = 100 * df.isnull().sum() / len(df)
    
    # sort in ascending order
    return percentage[percentage > 0].sort_values()

percent_missing_data = missing_data(clean_data)

sns.barplot(x = percent_missing_data.index, y = percent_missing_data)
plt.title('Percentage of missing data per feature')
plt.xlabel('Feature')
plt.ylabel('Percent of missing values (%)')
plt.xticks(rotation = 90)
plt.show()

### Chosing how to deal with missing values

There are the following options/steps:

1. Delete each row (sample) that contains a missing value
2. Delete the whole column (feature) containing the missing values
3. Replace the missing values with some other value

**Option 1**

Delete each row(sample) that contains a missing value.

In [ ]:
# display rows that miss more than 1% of the data
sns.barplot(x = percent_missing_data.index, y = percent_missing_data)
plt.xticks(rotation = 90)
plt.ylim(0 , 1)

# set labels
plt.title('Percentage of missing data per feature')
plt.xlabel('Feature')
plt.ylabel('Percent of missing values (%)')
plt.show()

if not keggle:
    
    # drop rows that miss less than 1% of the data
    drop_rows = percent_missing_data[percent_missing_data <= 1]
    drop_rows = drop_rows.index.tolist()
    clean_data = clean_data.dropna(axis = 'index', subset = drop_rows)

**Option 2**

Delete the whole column (feature) containing the missing values

In [ ]:
# select the features with more than 20% data missing
drop_features = percent_missing_data[percent_missing_data >= 20]
drop_features = drop_features.index.tolist()

# remove those features
clean_data = clean_data.drop(drop_features, axis = 1)

**Option 3**

Replace the missing values with some other value. For **categorical data**, replace the missing value with 'none'.
For **numerical data**, replace the missing value with the average value of that feature.

First, single out the features that have missing values. 

In [ ]:
# select features that have missing values between 1 and 20 percent missing data
discuss_features = percent_missing_data[np.logical_and(percent_missing_data > 1, percent_missing_data < 20)]
discuss_features = discuss_features.index.tolist()

print(f"The following features have missing values: {', '.join(discuss_features)}\n")

### Inspect each numeric feature seperately and decide how to best replace its missing values.

**1. Lot Frontage**

In [ ]:
# compute means of lot frontage per neighborhood
lot_means = clean_data.groupby('Neighborhood')['Lot Frontage'].mean()

# replace missing values 
clean_data['Lot Frontage'] = clean_data.groupby('Neighborhood')['Lot Frontage'].transform(lambda val: val.fillna(val.mean()))

# for GrHill use Timber mean since it is the closest
clean_data[clean_data['Neighborhood'] == 'GrnHill'] = clean_data[clean_data['Neighborhood'] == 'GrnHill'].fillna(lot_means['Timber'])

# for Landmrk use Old Town mean since it is the closes
clean_data[clean_data['Neighborhood'] == 'Landmrk'] = clean_data[clean_data['Neighborhood'] == 'Landmrk'].fillna(lot_means['OldTown'])

# remove lot frontage from features to be discussed
discuss_features.remove('Lot Frontage')

In [ ]:
# define difference between building years of house and garage
dev = clean_data['Year Built'] - clean_data['Garage Yr Blt']
# print(dev)
# define the average difference
avg_dev = np.round((dev.sum() / len(dev)), 0)

print(avg_dev)

# fill the nan values with the building year house with the avg deviation
clean_data['Garage Yr Blt'] = clean_data['Garage Yr Blt'].fillna(clean_data['Year Built'] + avg_dev)

print(clean_data['Garage Yr Blt'][2260])


**Other?** TBD

In [ ]:
# replace missing values
def replace_values(df):
    """
    This function replaces the missing values in the dataframe. If the missing
    value is in a categorical feature, we replace the value for a 'None' string. 
    In this case, you can still use that feature. For the numeric features, we 
    replaced the missing values with the average number of that feature. 
    
    """
    for feature in df:
        
        # single out categorical features and replace missing value with none
        if clean_data.dtypes[feature] == object:
            clean_data[feature] = clean_data[feature].fillna('None')

        # single out numerical features and replace missing value with mean
        else: 
            clean_data[feature] = clean_data[feature].fillna(clean_data[feature].mean())
            
    return clean_data
        
# check if all missing values are replaced
clean_data = replace_values(clean_data)

## Create new features

**1. Price/square feet based on neighborhood**

In [ ]:
# compute price per square feet and add to data frame 
clean_data['Price/SF'] = clean_data['SalePrice'] / clean_data['Gr Liv Area']

# compute average price per square feet per neighborhood
avg_price = clean_data.groupby('Neighborhood')['Price/SF'].mean()

# make dictionary from series
avg_price = avg_price.to_dict()

# replace values in price per sf column with average price per neighborhood
clean_data['Price/SF'] = clean_data['Neighborhood'].map(avg_price)

**2. Garden**

In [ ]:
clean_data['Garden Area'] = clean_data['Lot Area'] - (clean_data['Garage Area'] + clean_data['1st Flr SF'])
display(clean_data)

**3. Age House**

In [ ]:
clean_data['Age House'] = clean_data['Yr Sold'] - clean_data['Year Built']
clean_data

**4. Total area house**

In [ ]:
clean_data['Total House Area'] = clean_data['Total Bsmt SF'] + clean_data['Gr Liv Area']
clean_data

**5. Total full bathrooms**

In [ ]:
clean_data['Total Full Bath'] = clean_data['Bsmt Full Bath'] + clean_data['Full Bath']
display(clean_data)

**6. Total bathrooms**

In [ ]:
clean_data['Total Bath'] = clean_data['Full Bath'] + clean_data['Bsmt Full Bath'] + clean_data['Bsmt Half Bath'] + clean_data['Half Bath']

display(clean_data)

**7. Average surface per neighborhood**

In [ ]:
avg_surface = clean_data.groupby('Neighborhood')['Gr Liv Area'].mean()

# make dictionary from series
avg_surface = avg_surface.to_dict()

# replace values in price per sf column with average price per neighborhood
clean_data['SF Difference'] = clean_data['Neighborhood'].map(avg_surface)

clean_data['SF Difference'] = clean_data['Gr Liv Area'] - clean_data['SF Difference']

display(clean_data)

**8. inflation rate**

????


## Make dummies from categorical data


In [ ]:
# display feature data types in cleaned data
clean_data.info()

In [ ]:
# split dataframe in numerical and categorical
cat_data = clean_data.select_dtypes(include = 'object')
num_data = clean_data.drop(cat_data, axis=1)

display(cat_data.head())
display(num_data.head())

In [ ]:
# transform categorical features to one-hot-encoding
cat_data = pd.get_dummies(cat_data, drop_first=True)
display(cat_data.head())

## Rescaling data

In [ ]:
# temporary remove houses unique ID's and target value Saleprice
# because these should not be rescaled
ID = num_data['PID']
target = num_data['SalePrice']
num_data = num_data.drop(['SalePrice', 'PID'], axis=1)

# transform Sale Price in log Sale Price
target = np.log10(target)

# feature scale numerical data using zscore 
# note that we do not want to normalize our one-hot encoded data as those are already within [0,1] range
# source: "https://stackoverflow.com/a/41713622"
num_data = num_data.apply(zscore)

In [ ]:
# merge categorical and numerical dataframe
num_data = pd.concat([num_data, target, ID], axis = 1)
clean_data = pd.concat([cat_data, num_data], axis = 1)

# display ditribution of data
display(clean_data.describe())

In [ ]:
# display info clean_data
clean_data.info()
clean_data.head()

## Splitting data in test and training data

In [ ]:
# extract training and test houses in clean_data with ID
train_ID = train_data['PID'].to_numpy()
test_ID = test_data['PID'].to_numpy()

# split dataframe into test and train data
train_data = clean_data[clean_data['PID'].isin(train_ID)]
test_data = clean_data[clean_data['PID'].isin(test_ID)]

In [ ]:
# drop PID's
clean_data = clean_data.drop('PID', axis=1)
num_data = num_data.drop('PID', axis=1)
train_data = train_data.drop('PID', axis=1)
test_data = test_data.drop('PID', axis=1)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

## Exporting data to csv files

In [ ]:
# export keggle cleaned train and test data to seperate csv file
#export final cleaned dataframe to csv file  
if keggle:
    clean_data.to_csv('data/keggle_data.csv', index=False)  
    train_data.to_csv('data/keggle_train_data.csv', index=False)
    test_data.to_csv('data/keggle_test_data.csv', index=False)

else:   
    # export final cleaned dataframe to csv file
    clean_data.to_csv('data/clean_data.csv', index=False)
    train_data.to_csv('data/train_clean_data.csv', index=False)
    test_data.to_csv('data/test_clean_data.csv', index=False)
    
    # export cleaned numerical and categorical data to seperate csv file
    num_data.to_csv('data/num_clean_data.csv', index=False)
    cat_data.to_csv('data/cat_clean_data.csv', index=False)